In [3]:
import sys
import csv
import heapq
import time

# Creating an empty graph
graph = {}

# Importing the CSV with driving distances and mapping them to the driving distances from each state to the other state
def import_driving_distances():
    with open('driving.csv', newline='') as csvfile:
        data = list(csv.reader(csvfile))
        states = data[0]
        for i in range(1, len(data)):
            for j in range(1, len(states)):
                state1 = states[j]
                state2 = data[i][0]
                distance = data[i][j]
                if int(distance) != -1:
                    graph.setdefault(state1, []).append(StateNode(state2, distance))
                    graph.setdefault(state2, []).append(StateNode(state1, distance))

# Creating a priority queue class for the frontier
class PriorityQueue:
    def __init__(self):
        self.states = []

    def push(self, state, total_distance):
        heapq.heappush(self.states, (total_distance, state))

    def pop(self):
        return heapq.heappop(self.states)[1]

    def is_empty(self):
        return self.states == []

# Creating a state node. Each state has a distance
class StateNode:
    def __init__(self, state, distance):
        self.state = str(state)
        self.distance = str(distance)

#mapping the straight line distances 
def huristicdict(destination):

    #create an empty set s
    s = {}

    #open the straight line distance csv file
    with open("straightline.csv", 'r') as file:

        #read the file as a list
        data = list(csv.reader(file))

        #variable to keep count
        c = 0

        #store the names of the states as a list
        states = data[0]
        for st in states:
            if(st == destination):
                break
            c += 1
        
        #add the distances to the corresponding states
        if(c + 1 < len(data)):
          for i in range(1, len(data)):
                for _ in range (1, len(states)):
                    node = data[i][0]
                    straight_line_dist = data[i][c]
                    s[node] = straight_line_dist
    return s


# Returns the heuristic distances  
def calculate_heuristic(node, heuristic_values):
    return heuristic_values[node]

# Implementing the Greedy Best First Search algorithm
def greedy_best_first_search(source, destination, start_time):
    path = {}
    distance = {}
    priority_queue = PriorityQueue()
    heuristic_values = huristicdict(destination)
    visited = []
    expanded_nodes_count = 0  #expanded nodes

    priority_queue.push(source, 0)
    visited.append(source)
    distance[source] = 0
    path[source] = None

    while not priority_queue.is_empty():
        current = priority_queue.pop()
        if current == destination:
            break
        neighbors = {}
        min_state = current
        min_path = 100000
        min_distance = 10000
        count = 0
        expanded_nodes_count += 1  # expanded node
        if current in graph and len(heuristic_values) > 1:
            for new_state in graph[current]:
                if count < len(graph[current]):
                    count += 1
                    if new_state.state not in visited:
                        heuristic_distance = calculate_heuristic(new_state.state, heuristic_values)
                        neighbors[new_state.state] = heuristic_distance
                        visited.append(new_state.state)
                        if int(heuristic_distance) < int(min_path):
                            min_path = heuristic_distance
                            min_state = new_state.state
                            min_distance = new_state.distance
        if min_state not in distance:
            priority_queue.push(min_state, min_distance)
            distance[min_state] = int(min_distance)

    display_best_first_search(source, destination, path, distance, start_time)
    print("Number of expanded nodes:", expanded_nodes_count)  # Display the counter value after the search

#print output for greedy bfs
def display_best_first_search(source, destination, path, distance,start_time):
    finalpath = []
    i = destination

    while (path.get(i) != None):
        finalpath.append(i)
        i = path[i]
    finalpath.append(source)
    finalpath.reverse()
    print("Kumar, Aman, A20538809")
    print("Initial state: " + source)
    print("Goal state: " + destination)
    print("Greedy Best First Search:")

    if(distance.__contains__(destination) and distance.__contains__(source) and distance.__contains__(destination)):
          values = distance.values()
          total = sum(values)
          print("Solution path: " + str(list(distance.keys())))
          print("Number of states on a path: " + str(len(list(distance))))
          print("Path cost: " + str(total))
          print("Execution time "+str(time.time()-start_time))
    else :
         print("path not found")


# Implementing the A* search algorithm
def a_star_search(source, destination, start_time):
    path = {}
    distance = {}
    priority_queue = PriorityQueue()
    heuristic_values = huristicdict(destination)
    expanded_nodes_count = 0  # Initialize the counter for expanded nodes

    priority_queue.push(source, 0)
    distance[source] = 0
    path[source] = None

    while not priority_queue.is_empty():
        current_state = priority_queue.pop()
        if current_state == destination:
            break
        update_neighbors(current_state, path, distance, heuristic_values, priority_queue)
        expanded_nodes_count += 1  #  expanded node

    display_a_star_search(source, destination, path, distance, start_time)
    print("Number of expanded nodes:", expanded_nodes_count)  # Display the counter value after the search

#Updating the Neighbors
def update_neighbors(current_state, path, distance, heuristic_values, priority_queue):
    if current_state in graph and len(heuristic_values) > 1:
        for new_node in graph[current_state]:
            g_cost = distance[current_state] + int(new_node.distance)
            update_distance_and_path(current_state, new_node, g_cost, path, distance, heuristic_values, priority_queue)

#Updating distance and Path
def update_distance_and_path(current_state, new_node, g_cost, path, distance, heuristic_values, priority_queue):
    if new_node.state not in distance or g_cost < distance[new_node.state]:
        distance[new_node.state] = g_cost
        final_cost = g_cost + int(calculate_heuristic(new_node.state, heuristic_values))
        priority_queue.push(new_node.state, final_cost)
        path[new_node.state] = current_state

# Displaying output for A* search algorithm
def display_a_star_search(source, destination, path, distance, start_time):
    final_path = []
    i = destination

    while (path.get(i) != None):
        final_path.append(i)
        i = path[i]
    final_path.append(source)
    final_path.reverse()

    print("A* Search")
    if destination in distance and source in distance:
        print("Solution path:", final_path)
        print("Number of states on a path:", len(final_path))
        print("Path cost:", distance[destination])
        print("Execution time:", time.time() - start_time)
    else:
        print("Path not found")


def main():
    try:
        source = "IL"   #sys.argv[1]
        destination = "CA"   #sys.argv[2]

        if len(sys.argv) > 3:
            print("Error: Not enough or too many input arguments")
            exit()

        source = source.upper()
        destination = destination.upper()
    
    except IndexError:
        print("Error: Not enough or too many input arguments")
        exit()

    import_driving_distances()
    greedy_best_first_search(source, destination, time.time())
    a_star_search(source, destination, time.time())

if __name__ == "__main__":
    main()


FileNotFoundError: [Errno 2] No such file or directory: 'driving.csv'